# Part 1

In [36]:
import cv2
import numpy as np
from scipy.spatial import distance
from sklearn.cluster import KMeans

In [37]:
# Step 1: Load both images, convert to double and to grayscale
image1 = cv2.imread('left.jpg')
image2 = cv2.imread('right.jpg')

gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

In [38]:
# Step 2: Detect feature points in both images
sift = cv2.xfeatures2d.SIFT_create()

keypoints1, descriptors1 = sift.detectAndCompute(gray1, None)
keypoints2, descriptors2 = sift.detectAndCompute(gray2, None)

In [39]:
# Step 3: Compute distances between every descriptor in one image and every descriptor in the other image
descriptor_distances = distance.cdist(descriptors1, descriptors2, 'sqeuclidean')

In [40]:
# Step 4: Select putative matches based on the matrix of pairwise descriptor distances
# Here, we'll select the top 500 descriptor pairs with the smallest pairwise distances
num_matches = 500
indices = np.argsort(descriptor_distances, axis=None)[:num_matches]
matches = [(index // descriptor_distances.shape[1], index % descriptor_distances.shape[1]) for index in indices]

In [41]:
# Step 5: Implement RANSAC to estimate a homography mapping one image onto the other
def ransac(matches, keypoints1, keypoints2, threshold=5, iterations=1000):
    best_inliers = []
    best_homography = None
    best_residual = float('inf')

    for _ in range(iterations):
        # Randomly sample 4 matches
        sample_indices = np.random.choice(len(matches), 4, replace=False)
        sampled_matches = [matches[i] for i in sample_indices]

        # Estimate homography
        src_pts = np.float32([keypoints1[m[0]].pt for m in sampled_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints2[m[1]].pt for m in sampled_matches]).reshape(-1, 1, 2)
        homography, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, threshold)

        # Count inliers
        inliers = []
        for match in matches:
            src_pt = np.float32(keypoints1[match[0]].pt).reshape(-1, 1, 2)
            dst_pt = np.float32(keypoints2[match[1]].pt).reshape(-1, 1, 2)
            transformed_pt = cv2.perspectiveTransform(src_pt, homography)
            error = np.linalg.norm(dst_pt - transformed_pt)
            if error < threshold:
                inliers.append(match)

        # Update best model if necessary
        if len(inliers) > len(best_inliers):
            best_inliers = inliers
            best_homography = homography
            best_residual = np.mean([np.linalg.norm(np.float32(keypoints2[m[1]].pt).reshape(-1, 1, 2) - cv2.perspectiveTransform(np.float32(keypoints1[m[0]].pt).reshape(-1, 1, 2), best_homography)) for m in best_inliers])

    return best_homography, best_inliers, best_residual

In [42]:
homography, inliers, residual = ransac(matches, keypoints1, keypoints2)

In [43]:
# Report the number of inliers and the average residual for the inliers
print("Number of inliers:", len(inliers))
print("Average residual:", residual)

Number of inliers: 107
Average residual: 1.4440694


In [44]:
# Display the locations of inlier matches in both images
result = cv2.drawMatches(image1, keypoints1, image2, keypoints2, [cv2.DMatch(i, i, 0) for i in range(len(inliers))], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

cv2.imwrite("Matches.jpg", result)
#cv2.imshow("Matches", result)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

True

# Part 2

In [45]:
# Warp image1 onto image2 using the estimated homography
h, w = image2.shape[:2]
warped_image = cv2.warpPerspective(image1, homography, (w, h))

In [46]:
# Composite the two images by averaging pixel values
# You can also use other blending techniques here
composite_image = (warped_image + image2) / 2

In [48]:
# Save or display the resulting composite image
cv2.imwrite("composite_image.jpg", composite_image)

True